# IFC OpenShell Voice Assistant - Prototype #5


#### Example code from
https://academy.ifcopenshell.org/posts/creating-a-simple-wall-with-property-set-and-quantity-information/

This code creates an ifc template file, a wall element, and an openning in it

In [1]:
import uuid
import time
import tempfile
import ifcopenshell

In [2]:
O = 0., 0., 0.
X = 1., 0., 0.
Y = 0., 1., 0.
Z = 0., 0., 1.

In [3]:
# Helper function definitions

# Creates an IfcAxis2Placement3D from Location, Axis and RefDirection specified as Python tuples
def create_ifcaxis2placement(ifcfile, point=O, dir1=Z, dir2=X):
    point = ifcfile.createIfcCartesianPoint(point)
    dir1 = ifcfile.createIfcDirection(dir1)
    dir2 = ifcfile.createIfcDirection(dir2)
    axis2placement = ifcfile.createIfcAxis2Placement3D(point, dir1, dir2)
    return axis2placement

# Creates an IfcLocalPlacement from Location, Axis and RefDirection, specified as Python tuples, and relative placement
def create_ifclocalplacement(ifcfile, point=O, dir1=Z, dir2=X, relative_to=None):
    axis2placement = create_ifcaxis2placement(ifcfile,point,dir1,dir2)
    ifclocalplacement2 = ifcfile.createIfcLocalPlacement(relative_to,axis2placement)
    return ifclocalplacement2

# Creates an IfcPolyLine from a list of points, specified as Python tuples
def create_ifcpolyline(ifcfile, point_list):
    ifcpts = []
    for point in point_list:
        point = ifcfile.createIfcCartesianPoint(point)
        ifcpts.append(point)
    polyline = ifcfile.createIfcPolyLine(ifcpts)
    return polyline
    
# Creates an IfcExtrudedAreaSolid from a list of points, specified as Python tuples
def create_ifcextrudedareasolid(ifcfile, point_list, ifcaxis2placement, extrude_dir, extrusion):
    polyline = create_ifcpolyline(ifcfile, point_list)
    ifcclosedprofile = ifcfile.createIfcArbitraryClosedProfileDef("AREA", None, polyline)
    ifcdir = ifcfile.createIfcDirection(extrude_dir)
    ifcextrudedareasolid = ifcfile.createIfcExtrudedAreaSolid(ifcclosedprofile, ifcaxis2placement, ifcdir, extrusion)
    return ifcextrudedareasolid
    
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)


In [4]:
# IFC template creation

filename = "hello_wall_P_5.ifc"
timestamp = time.time()
timestring = time.strftime("%Y-%m-%dT%H:%M:%S", time.gmtime(timestamp))
creator = "Kianwee Chen"
organization = "RWTH"
application, application_version = "IfcOpenShell", "0.5"
project_globalid, project_name = create_guid(), "Hello Wall"
    
# A template IFC file to quickly populate entity instances for an IfcProject with its dependencies
template = """ISO-10303-21;
HEADER;
FILE_DESCRIPTION(('ViewDefinition [CoordinationView]'),'2;1');
FILE_NAME('%(filename)s','%(timestring)s',('%(creator)s'),('%(organization)s'),'%(application)s','%(application)s','');
FILE_SCHEMA(('IFC2X3'));
ENDSEC;
DATA;
#1=IFCPERSON($,$,'%(creator)s',$,$,$,$,$);
#2=IFCORGANIZATION($,'%(organization)s',$,$,$);
#3=IFCPERSONANDORGANIZATION(#1,#2,$);
#4=IFCAPPLICATION(#2,'%(application_version)s','%(application)s','');
#5=IFCOWNERHISTORY(#3,#4,$,.ADDED.,$,#3,#4,%(timestamp)s);
#6=IFCDIRECTION((1.,0.,0.));
#7=IFCDIRECTION((0.,0.,1.));
#8=IFCCARTESIANPOINT((0.,0.,0.));
#9=IFCAXIS2PLACEMENT3D(#8,#7,#6);
#10=IFCDIRECTION((0.,1.,0.));
#11=IFCGEOMETRICREPRESENTATIONCONTEXT($,'Model',3,1.E-05,#9,#10);
#12=IFCDIMENSIONALEXPONENTS(0,0,0,0,0,0,0);
#13=IFCSIUNIT(*,.LENGTHUNIT.,$,.METRE.);
#14=IFCSIUNIT(*,.AREAUNIT.,$,.SQUARE_METRE.);
#15=IFCSIUNIT(*,.VOLUMEUNIT.,$,.CUBIC_METRE.);
#16=IFCSIUNIT(*,.PLANEANGLEUNIT.,$,.RADIAN.);
#17=IFCMEASUREWITHUNIT(IFCPLANEANGLEMEASURE(0.017453292519943295),#16);
#18=IFCCONVERSIONBASEDUNIT(#12,.PLANEANGLEUNIT.,'DEGREE',#17);
#19=IFCUNITASSIGNMENT((#13,#14,#15,#18));
#20=IFCPROJECT('%(project_globalid)s',#5,'%(project_name)s',$,$,$,$,(#11),#19);
ENDSEC;
END-ISO-10303-21;
""" % locals()

# Write the template to a temporary file 
temp_handle, temp_filename = tempfile.mkstemp(suffix=".ifc")
with open(temp_filename, "w+") as f:
    f.write(template)
 
# Obtain references to instances defined in template
ifcfile = ifcopenshell.open(temp_filename)
owner_history = ifcfile.by_type("IfcOwnerHistory")[0]
project = ifcfile.by_type("IfcProject")[0]
context = ifcfile.by_type("IfcGeometricRepresentationContext")[0]


In [5]:
# IFC hierarchy creation
site_placement = create_ifclocalplacement(ifcfile)
site = ifcfile.createIfcSite(create_guid(), owner_history, "Site", None, None, site_placement, None, None, "ELEMENT", None, None, None, None, None)

building_placement = create_ifclocalplacement(ifcfile, relative_to=site_placement)
building = ifcfile.createIfcBuilding(create_guid(), owner_history, 'Building', None, None, building_placement, None, None, "ELEMENT", None, None, None)

storey_placement = create_ifclocalplacement(ifcfile, relative_to=building_placement)
elevation = 0.0
building_storey = ifcfile.createIfcBuildingStorey(create_guid(), owner_history, 'Storey', None, None, storey_placement, None, None, "ELEMENT", elevation)

container_storey = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Building Container", None, building, [building_storey])
container_site = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Site Container", None, site, [building])
container_project = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Project Container", None, project, [site])


In [6]:
# Wall creation: Define the wall shape as a polyline axis and an extruded area solid
wall_placement = create_ifclocalplacement(ifcfile, relative_to=storey_placement)
polyline = create_ifcpolyline(ifcfile, [(0.0, 0.0, 0.0), (5.0, 0.0, 0.0)])
axis_representation = ifcfile.createIfcShapeRepresentation(context, "Axis", "Curve2D", [polyline])

extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
point_list_extrusion_area = [(0.0, -0.1, 0.0), (5.0, -0.1, 0.0), (5.0, 0.1, 0.0), (0.0, 0.1, 0.0), (0.0, -0.1, 0.0)]
solid = create_ifcextrudedareasolid(ifcfile, point_list_extrusion_area, extrusion_placement, (0.0, 0.0, 1.0), 3.0)
body_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [solid])

product_shape = ifcfile.createIfcProductDefinitionShape(None, None, [axis_representation, body_representation])

wall = ifcfile.createIfcWallStandardCase(create_guid(), owner_history, "Wall", "An awesome wall", None, wall_placement, product_shape, None)

# Define and associate the wall material
material = ifcfile.createIfcMaterial("wall material")
material_layer = ifcfile.createIfcMaterialLayer(material, 0.2, None)
material_layer_set = ifcfile.createIfcMaterialLayerSet([material_layer], None)
material_layer_set_usage = ifcfile.createIfcMaterialLayerSetUsage(material_layer_set, "AXIS2", "POSITIVE", -0.1)
ifcfile.createIfcRelAssociatesMaterial(create_guid(), owner_history, RelatedObjects=[wall], RelatingMaterial=material_layer_set_usage)

# Create and assign property set
property_values = [
    ifcfile.createIfcPropertySingleValue("Reference", "Reference", ifcfile.create_entity("IfcText", "Describe the Reference"), None),
    ifcfile.createIfcPropertySingleValue("IsExternal", "IsExternal", ifcfile.create_entity("IfcBoolean", True), None),
    ifcfile.createIfcPropertySingleValue("ThermalTransmittance", "ThermalTransmittance", ifcfile.create_entity("IfcReal", 2.569), None),
    ifcfile.createIfcPropertySingleValue("IntValue", "IntValue", ifcfile.create_entity("IfcInteger", 2), None)
]
property_set = ifcfile.createIfcPropertySet(create_guid(), owner_history, "Pset_WallCommon", None, property_values)
ifcfile.createIfcRelDefinesByProperties(create_guid(), owner_history, None, None, [wall], property_set)

# Add quantity information
quantity_values = [
    ifcfile.createIfcQuantityLength("Length", "Length of the wall", None, 5.0),
    ifcfile.createIfcQuantityArea("Area", "Area of the front face", None, 5.0 * solid.Depth),
    ifcfile.createIfcQuantityVolume("Volume", "Volume of the wall", None, 5.0 * solid.Depth * material_layer.LayerThickness)
]
element_quantity = ifcfile.createIfcElementQuantity(create_guid(), owner_history, "BaseQuantities", None, None, quantity_values)
ifcfile.createIfcRelDefinesByProperties(create_guid(), owner_history, None, None, [wall], element_quantity)


#82=IfcRelDefinesByProperties('3TfrEpUMGHxB03mBY3dDM_',#5,$,$,(#66),#81)

In [7]:
# Create and associate an opening for the window in the wall
opening_placement = create_ifclocalplacement(ifcfile, (0.5, 0.0, 1.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), wall_placement)
opening_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
point_list_opening_extrusion_area = [(0.0, -0.1, 0.0), (3.0, -0.1, 0.0), (3.0, 0.1, 0.0), (0.0, 0.1, 0.0), (0.0, -0.1, 0.0)]
opening_solid = create_ifcextrudedareasolid(ifcfile, point_list_opening_extrusion_area, opening_extrusion_placement, (0.0, 0.0, 1.0), 1.0)
opening_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [opening_solid])
opening_shape = ifcfile.createIfcProductDefinitionShape(None, None, [opening_representation])
opening_element = ifcfile.createIfcOpeningElement(create_guid(), owner_history,"Opening", "An awesome opening", None, opening_placement, opening_shape, None)
ifcfile.createIfcRelVoidsElement(create_guid(), owner_history, None, None, wall, opening_element)

# Create a simplified representation for the Window
window_placement = create_ifclocalplacement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), opening_placement)
window_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
point_list_window_extrusion_area = [(0.0, -0.01, 0.0), (3.0, -0.01, 0.0), (3.0, 0.01, 0.0), (0.0, 0.01, 0.0), (0.0, -0.01, 0.0)]
window_solid = create_ifcextrudedareasolid(ifcfile, point_list_window_extrusion_area, window_extrusion_placement, (0.0, 0.0, 1.0), 1.0)
window_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [window_solid])
window_shape = ifcfile.createIfcProductDefinitionShape(None, None, [window_representation])
window = ifcfile.createIfcWindow(create_guid(), owner_history, "Window", "An awesome window", None, window_placement, window_shape, None, None)


In [8]:
# Relate the window to the opening element
ifcfile.createIfcRelFillsElement(create_guid(), owner_history, None, None, opening_element, window)

# Relate the window and wall to the building storey
ifcfile.createIfcRelContainedInSpatialStructure(create_guid(), owner_history, "Building Storey Container", None, [wall, window], building_storey)

# Write the contents of the file to disk
ifcfile.write(filename)
# print(open(filename, 'r').read()) 

##### Voice assistant prototype #5

In [26]:
from IPython.core.display import Javascript
Javascript(var bimSurfer = new BimSurfer({domNode: "viewerContainer"}))

SyntaxError: invalid syntax (<ipython-input-26-c803d718e3b3>, line 2)

In [5]:
%%html
<iframe src="https://ifcjs.github.io/web-ifc-viewer/example/index" width="500" height="250"></iframe>

In [9]:
# import this modules for ifcopenshell commands and voice recognition
import ifcopenshell
import numpy
import apiai
import speech_recognition as sr
import re

In [10]:
# Run and Google will start to listen to you, tell us what you want to do 
def get_voice():
    raw_text = {}
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        r.adjust_for_ambient_noise(source, duration=1)
        r.pause_threshold = 0.8
        r.dynamic_energy_threshold = True
        audio = r.listen(source)
        raw_text = r.recognize_google(audio)
    return raw_text

In [20]:
# Run and Google will start to listen you, tell us what you want to do 
def get_voice():
    raw_text = {}
    r = sr.Recognizer()
    mic = sr.Microphone()
    tts (f' How can I help you?')
    # x = False
    # while x == False:
        # tts (f' Sorry, but I could not understand your command, can you repeat it?')
        # try:
        with mic as source:
            r.adjust_for_ambient_noise(source, duration=1)
            r.pause_threshold = 0.8
            r.dynamic_energy_threshold = True
            audio = r.listen(source)
            raw_text = r.recognize_google(audio)
            x = bool([x for x in raw_text if x in sensitive_keywords])


        # except:
        #     tts (f' Sorry, but I could not understand your command, can you repeat it?')
        #     with mic as source:
        #         r.adjust_for_ambient_noise(source, duration=1)
        #         r.pause_threshold = 0.8
        #         r.dynamic_energy_threshold = True
        #         audio = r.listen(source)
        #         raw_text = r.recognize_google(audio)
        # elif x == False:
        #     break

        # else:
        #     tts (f' Sorry, but I could not understand your command, can you repeat it?')
        #     with mic as source:
        #         r.adjust_for_ambient_noise(source, duration=1)
        #         r.pause_threshold = 0.8
        #         r.dynamic_energy_threshold = True
        #         audio = r.listen(source)
        #         raw_text = r.recognize_google(audio)
        # elif x == False:
        # break

        # finally:
        #         tts (f' Sorry, but I could not understand your command, can you repeat it?')
        #         with mic as source:
        #             r.adjust_for_ambient_noise(source, duration=1)
        #             r.pause_threshold = 0.8
        #             r.dynamic_energy_threshold = True
        #             audio = r.listen(source)
        #             raw_text = r.recognize_google(audio)
        # if x == False:
        #     break
        
    return raw_text

In [22]:
get_voice()


UnknownValueError: 

In [36]:
[x for x in raw_text if x in sensitive_keywords]

[]

In [12]:
# // new speech recognition object
# var SpeechRecognition = SpeechRecognition || webkitSpeechRecognition;
# var recognition = new SpeechRecognition();
            
# // This runs when the speech recognition service starts
# recognition.onstart = function() {
#     console.log("We are listening. Try speaking into the microphone.");
# };

# recognition.onspeechend = function() {
#     // when user is done speaking
#     recognition.stop();
# }
              
# // This runs when the speech recognition service returns result
# recognition.onresult = function(event) {
#     var transcript = event.results[0][0].transcript;
#     var confidence = event.results[0][0].confidence;
# };
              
# // start recognition
# recognition.start();

In [5]:
import pyttsx3;
def tts (text):
    engine = pyttsx3.init();
    engine.say(text);
    # engine.setProperty('voice', voices[2].id)
    engine.setProperty("rate", 155)
    engine.runAndWait() ;

In [14]:
name = 'Saeed'

open an ifc file

In [15]:
tts (f' , How can I help you {name}')

In [20]:
command_text = get_voice()
command_text

'open at 5'

In [6]:
sensitive_keywords = ["wall", "door", "open","create","change", "name", "height", "y dimension", "x dimension", "description", "global id" "Red","Blue","Black"]
colors = ["red","blue","black"]
dimensions = ["x","y","height"]
colors = ["red","blue","black"]
elements = ["wall","window","door"]
files = ["4","5","6"]

In [22]:
command_ = [x for x in sensitive_keywords if x in command_text]
bool(command_)

True

In [23]:
tts (f' Ok, I will {command_}')

In [24]:
file_name = "ifcmodeltext.txt"
ifc_model_text = open("{file_name}.txt", "a+")

In [48]:
print(file_name)

what is 1.6


In [47]:
import webbrowser
webbrowser.open(file_name)

True

In [26]:
files

['4', '5', '6']

In [32]:
file_name = get_voice()
print(file_name)
file_name_ = re.findall('\d*\.?\d+', file_name)[0]
print(file_name_)
bool([x for x in file_name_ if x in files])

what is 1.6
1.6


True

In [81]:
def file_opening():
    
    # file_name = {}
    # if bool(file_name) == False:
    tts (f' Can you please give me the file name')
    get_voice_text = get_voice()
    file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]
    if [x for x in file_name_ if x in files]:
        # print(x)
        webbrowser.open(f'{file_name_}.txt')
    else:
        tts (f' Sorry, but I can only open file {files}')

            
    print(file_name_)

        

In [83]:
file_opening()

7


In [94]:
import speech_recognition as sr
def callback(recognizer, audio):                          # this is called from the background thread
    try:
        print("You said " + recognizer.recognize(audio))  # received audio data, now need to recognize it
    except LookupError:
        print("Oops! Didn't catch that")

report the name of the elements

In [66]:
command_text = get_voice()
command_text
command_ = [x for x in sensitive_keywords if x in command_text]
tts (f' Ok, I will give you the {command_}')


NameError: name 'commands_' is not defined

In [67]:
print (command_)

['change', 'height', 'y']


In [44]:
wall_name = wall.Name

In [45]:
tts (f' the name of the element is {wall_name}')

report the description of the elements

In [46]:
command_text = get_voice()
command_text
command_ = [x for x in sensitive_keywords if x in command_text]
tts (f' Ok, I will give you the {command_}')

In [47]:
wall_description = wall.Description
tts (f' wall description is {wall_description}')

create an ifc file


In [64]:
command_text = get_voice()
print(command_text)
# command_text.extract('(\d+\W*\d*)')
# command_ = [x for x in command_text if x.isdigit()]
# command_
import re
value_ = re.findall('\d*\.?\d+',command_text)[0]
value_
tts (f' Ok, value you gave me was {value_}, right?')


the value is 1.8


create a wall

create a window

create a door

change the color of objects

change the size of the viewer

In [ ]:
ifcfile = ifcopenshell.open("./one.ifc")

In [43]:
import ifcopenshell
import numpy
import apiai
import speech_recognition as sr
import re
import pyttsx3
import webbrowser


sensitive_keywords = ["colour", "dimension", "element", "file"]
colours = ["red","blue","yellow"]
dimensions = ["x","y","height"]


elements = ["wall","window","door"]
files = ["4","5","6"]


def text_to_voice (text):

    engine = pyttsx3.init();
    engine.say(text);
    # engine.setProperty('voice', voices[2].id)
    engine.setProperty("rate", 155)
    engine.runAndWait() ;


def get_voice():

    raw_text = {}
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        r.adjust_for_ambient_noise(source, duration=1)
        r.pause_threshold = 0.8
        r.dynamic_energy_threshold = True
        audio = r.listen(source)
        raw_text = r.recognize_google(audio)
    return raw_text


def nlp(raw_text, sensitive_keywords):

    command_type = [x for x in sensitive_keywords if x in raw_text]
    if command_type[0] == 'colour':
        command_sup = [x for x in colours if x in raw_text]
    elif command_type[0] == 'dimension':
        command_sup = [x for x in dimensions if x in raw_text]
    elif command_type[0] == 'element':
        command_sup = [x for x in elements if x in raw_text]
    elif command_type[0] == 'file':
        command_sup = [x for x in files if x in raw_text]
    return command_type, command_sup


def open_file():

    text_to_voice (f' Can you please give me the file name')
    get_voice_text = get_voice()
    file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]

    if [x for x in file_name_ if x in files]:
        
        webbrowser.open(f'{file_name_}.txt')

    else:
        
        while bool([x for x in file_name_ if x in files])==False:
            
            text_to_voice (f' Sorry, but I can only open file {files}, Can you please give me the file name?')
            get_voice_text = get_voice()
            file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]
            if [x for x in file_name_ if x in files]:
                webbrowser.open(f'{file_name_}.txt')


def get_action(command_type, command_sup):

    if command_type[0] == 'colour':
        pass
    elif command_type[0] == 'dimension':
        pass
    elif command_type[0] == 'element':
        pass
    elif command_type[0] == 'file':
        open_file()           




In [38]:
text_to_voice ('asadsad')

In [39]:
raw_text = get_voice()


In [40]:
raw_text

'please open a file for me'

In [41]:
command_type, command_sup = nlp(raw_text, sensitive_keywords)

In [48]:
get_action(command_type, command_sup)
